In [1]:
import compas.datastructures.mesh.subdivision as sd
import ipyvolume as ipv
from utilities import draw_compas_mesh, export_obj_by_attribute
from shapes import *
from compas.datastructures import Mesh, meshes_join
from compas.geometry import distance_point_point, Vector, Circle
from statistics import median, variance


origin = (0,0,0)
normal = (0,0,1)
plane = (origin, normal)
r_axis = 5
r_pipe = .5

torus = Torus(plane, r_axis, r_pipe)

vs, fs = torus.to_vertices_and_faces(u = 30, v = 6)

mesh = Mesh.from_vertices_and_faces(vs, fs)

mesh = sd.mesh_subdivide_catmullclark(mesh, k=1)
mesh = sd.mesh_subdivide_doosabin(mesh)

#draw_compas_mesh(subd)

draw_compas_mesh(mesh)
#mesh = subd

Mesh(color=array('white', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [2]:
distances = []

for v in mesh.vertices():
    v_coord = mesh.vertex_coordinates(v)
    distance_to_mid = distance_point_point(origin, v_coord)
    distances.append(distance_to_mid)



In [3]:
distance_outer_ring = round(max(distances), 1)
distance_inner_ring = round(min(distances), 1)

print(distance_inner_ring)
print(distance_outer_ring)

4.5
5.4


In [4]:
point_list = []

# distance to move outer and inner vertex
distance = 50

for v in mesh.vertices():
    v_coord = mesh.vertex_coordinates(v)
    point_list.append(v_coord)
    distance_to_mid = distance_point_point(origin, v_coord)
     
    if distance_to_mid == distance_outer_ring:
        mesh.set_vertex_attribute(v, 'location', 'outer')
    else:
        mesh.set_vertex_attribute(v, 'location', 'inner')



for vk in mesh.vertices():
    vtype = mesh.get_vertex_attribute(vk, 'location')
    if vtype:
        v_coord = mesh.vertex_coordinates(vk)
        move_vector = Vector.from_start_end(origin, v_coord)
        move_vector.unitize()
        move_vector.__imul__(distance)
        
        if vtype == 'outer':
   
            new_coord = Vector.sum_vectors([v_coord, move_vector])
            mesh.vertex[vk]['x'] = new_coord[0]
            mesh.vertex[vk]['y'] = new_coord[1]
            mesh.vertex[vk]['z'] = new_coord[2]
        elif vtype == 'inner':
            new_coord = Vector.sum_vectors([v_coord, move_vector])
            mesh.vertex[vk]['x'] = new_coord[0]
            mesh.vertex[vk]['y'] = new_coord[1]
            mesh.vertex[vk]['z'] = new_coord[2]
            

draw_compas_mesh(mesh)

Mesh(color=array('white', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [5]:
distances = []
for fkey in mesh.faces():
    f_centroid = mesh.face_centroid(fkey)
    distances.append(distance_point_point(origin, f_centroid))


median_distance = median(distances)
median_distance = round(median_distance, 10)

variance_dist = variance(distances)

In [6]:
for fkey in mesh.faces():
    f_centroid = mesh.face_centroid(fkey)
    distance = distance_point_point(origin, f_centroid)
    distance = round(distance, 9)
    
    if distance > median_distance + variance_dist:
        mesh.set_face_attribute(fkey, 'location', 'outer')
    elif distance < median_distance - variance_dist:
        mesh.set_face_attribute(fkey, 'location', 'inner')
    else:
        mesh.set_face_attribute(fkey, 'location', 'sides')

In [7]:
export_obj_by_attribute('ring_1.obj', mesh, 'location')
mesh.to_json('ring_1.json', pretty=True)